# Demo : ETL Elasticsearch (Extract)

Import libraries. At this point, you should already runs previous lessons and have the libraries installed.

In [ ]:
import psycopg2

Open connection to postgresql

In [ ]:
try:
    conn = psycopg2.connect("host=34.101.229.192 dbname=postgres user=postgres password=CourseDE888")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
except Exception as e:
    print("Error: cannot open cursor for SQL interaction")
    print(e)

Clear staging table

In [ ]:
cur.execute("""
    DELETE FROM staging.stg_es_movie_principals
           WHERE date_trunc('day', created_date) = date_trunc('day', now())
""")

Populate data for staging tables

In [ ]:
sql_staging_movie_principals = """
    INSERT INTO staging.stg_es_movie_principals (
        SELECT
            m.title_id,
            m.title,
            m.original_title,
            m.duration_minutes,
            m.date_published,
            m.production_company,
            m.description,
            p.name_id,
            p.name,
            mpc.category,
            NULL AS job,
            NULL AS CHARACTER
        FROM
            movies m,
            people p,
            movie_principal_categories mpc
        WHERE
            m.movie_id = mpc.movie_id
            AND p.person_id = mpc.person_id
        UNION
        SELECT
            m.title_id,
            m.title,
            m.original_title,
            m.duration_minutes,
            m.date_published,
            m.production_company,
            m.description,
            p.name_id,
            p.name,
            NULL AS category,
            NULL AS job,
            mpch.character
        FROM
            movies m,
            people p,
            movie_principal_characters mpch
        WHERE
            m.movie_id = mpch.movie_id
            AND p.person_id = mpch.person_id
        UNION
        SELECT
            m.title_id,
            m.title,
            m.original_title,
            m.duration_minutes,
            m.date_published,
            m.production_company,
            m.description,
            p.name_id,
            p.name,
            NULL AS category,
            mj.job AS job,
            NULL AS CHARACTER
        FROM
            movies m,
            people p,
            movie_principal_jobs mj
        WHERE
            m.movie_id = mj.movie_id
            AND p.person_id = mj.person_id
    )
"""

In [ ]:
cur.execute(sql_staging_movie_principals)

Recheck staging table

In [ ]:
cur.execute("SELECT count(*) FROM staging.stg_es_movie_principals")
cur.fetchone()[0]